# Sub Question Query Engine
In this tutorial, we showcase how to use a **sub question query engine** to tackle the problem of answering a complex query using multiple data sources.  
It first breaks down the complex query into sub questions for each relevant data source,
then gather all the intermediate reponses and synthesizes a final response.

### Preparation

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [3]:
!pip install llama-index

In [36]:
import os
#REMOVE KEY
os.environ["OPENAI_API_KEY"] = ""

import nest_asyncio

nest_asyncio.apply()

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings

In [7]:
# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

In [8]:
!unzip /content/WeissRules.zip

Archive:  /content/WeissRules.zip
  inflating: Weiss Rules/Hololive Set 1 Cards HOL-xxx.txt  
  inflating: Weiss Rules/Official Comprehensive Rules.txt  
  inflating: Weiss Rules/Weissplaining 10 Encore The Step vs. The Keyword Ability.txt  
  inflating: Weiss Rules/Weissplaining 8 Do Things Stack.txt  
  inflating: Weiss Rules/Weissplaining 9Few Cards Left in Deck, Now What.txt  
  inflating: Weiss Rules/Weissplaining1 Playing vs. Placing On Stage.txt  
  inflating: Weiss Rules/Weissplaining2 The Resolution Zone.txt  
  inflating: Weiss Rules/Weissplaining3 Check Timing, Play Timing & Rule Actions.txt  
  inflating: Weiss Rules/Weissplaining4 Double Losses in Weiss aka Ifrit at Level 3.txt  
  inflating: Weiss Rules/Weissplaining5  Why Counters And Backups Arent The Same Thing.txt  
  inflating: Weiss Rules/Weissplaining6 Multiple AUTO Abilities Activating At The Same Time.txt  
  inflating: Weiss Rules/Weissplaining7 Reveal vs. Flip Over vs. Look At.txt  
  inflating: Weiss Rules/WTT

In [9]:
# load data
humberviewbook = SimpleDirectoryReader(input_dir="./WeissRules/").load_data()

# build index and query engine
vector_query_engine = VectorStoreIndex.from_documents(
    humberviewbook,
    use_async=True,
).as_query_engine()

**********
Trace: index_construction
    |_node_parsing -> 0.330148 seconds
      |_chunking -> 0.144634 seconds
      |_chunking -> 0.079674 seconds
      |_chunking -> 0.004007 seconds
      |_chunking -> 0.007886 seconds
      |_chunking -> 0.004859 seconds
      |_chunking -> 0.013984 seconds
      |_chunking -> 0.003175 seconds
      |_chunking -> 0.003454 seconds
      |_chunking -> 0.007782 seconds
      |_chunking -> 0.000719 seconds
      |_chunking -> 0.006689 seconds
      |_chunking -> 0.009687 seconds
      |_chunking -> 0.004501 seconds
    |_embedding -> 1.974869 seconds
    |_embedding -> 2.189581 seconds
**********


### Setup sub question query engine

In [10]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="JudgeBot",
            description="Provides ruling help on the TCG Weiss Schwarz.",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

### Run queries

In [35]:
response = query_engine.query(

    "You are a judge for the card game Weiss Schawrz. Questions will be about the game's rules and how situations are resolved. Please be as specific as possible. Question: If two cards are reversed at the same time and they both have on reverse effects, which player resolves their on reverse effect first?"
)

Generated 1 sub questions.
[JudgeBot] Q: Which player resolves their on reverse effect first when two cards are reversed at the same time?
[JudgeBot] A: The player whose turn it is gets to choose which of the two abilities to resolve first when two cards are reversed at the same time.
**********
Trace: query
    |_query -> 2.931244 seconds
      |_llm -> 0.944674 seconds
      |_sub_question -> 1.087131 seconds
        |_query -> 1.085904 seconds
          |_retrieve -> 0.153857 seconds
            |_embedding -> 0.120302 seconds
          |_synthesize -> 0.930972 seconds
            |_templating -> 4.6e-05 seconds
            |_llm -> 0.914411 seconds
      |_synthesize -> 0.894826 seconds
        |_templating -> 2.7e-05 seconds
        |_llm -> 0.890879 seconds
**********


In [31]:
print(response)

Layer resolves its on-reverse effect first when two cards are reversed at the same time.


In [29]:
# iterate through sub_question items captured in SUB_QUESTION event
from llama_index.core.callbacks import CBEventType, EventPayload

for i, (start_event, end_event) in enumerate(
    llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)
):
    qa_pair = end_event.payload[EventPayload.SUB_QUESTION]
    print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
    print("Answer: " + qa_pair.answer.strip())
    print("====================================")

Sub Question 0: What are the key strategies for winning in TCG Weiss Schwarz?
Answer: Understanding the card interactions, mastering the game mechanics, and building a well-balanced deck are key strategies for winning in TCG Weiss Schwarz. Additionally, being familiar with the rules, knowing how to effectively use card abilities, and adapting your gameplay based on the situation are important aspects to consider for success in the game.
Sub Question 1: What are the win conditions in TCG Weiss Schwarz?
Answer: If any player fulfills any losing conditions, that player loses the game by rule action the next time rule actions are performed. Winning conditions in TCG Weiss Schwarz include having the opponent lose while you do not, or winning or losing the game by a card effect during resolution.
Sub Question 2: What is the rule number for brainstorming in TCG Weiss Schwarz?
Answer: Rule number for brainstorming in TCG Weiss Schwarz is Section 8.
Sub Question 3: What are the rules for resolv